In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
chernobyl_coords = (30.0927, 51.3870) # (Lon, lat) or (x, y)

MAP_SIZE_IN_DEGREES = (100, 50)

LOWER_LEFT_X_LON = chernobyl_coords[0] - MAP_SIZE_IN_DEGREES[0]/2
LOWER_LEFT_Y_LAT = chernobyl_coords[1] - MAP_SIZE_IN_DEGREES[1]/2
UPPER_LEFT_X_LON = chernobyl_coords[0] + MAP_SIZE_IN_DEGREES[0]/2
UPPER_LEFT_Y_LAT = chernobyl_coords[1] + MAP_SIZE_IN_DEGREES[1]/2



In [ ]:
fig = plt.figure(figsize=(12,9))

# world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
m = Basemap(projection='mill',
            llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
            llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
            urcrnrlon=UPPER_LEFT_X_LON, 
            urcrnrlat=UPPER_LEFT_Y_LAT,
            resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

m.shadedrelief() # makes it look pretty :)
m.drawcountries(linewidth=1.2)

# Locate Chernobyl
m.scatter(chernobyl_coords[0], chernobyl_coords[1], latlon=True, s=100, c='red', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

plt.show()

In [ ]:
sensor_latitudes = []
sensor_longitudes = []

data = pd.read_csv("data/archive/data.csv")

for lon, lat in data.filter(['X','Y']).drop_duplicates().values:
    sensor_longitudes.append(lon)
    sensor_latitudes.append(lat)


assert len(sensor_latitudes) == len(sensor_longitudes) and len(sensor_latitudes) > 50

In [ ]:
fig = plt.figure(figsize=(12,9))

# world map has a lower left corner of (-180, -90) and an upper right corner of (180, 90)
m = Basemap(projection='mill',
            llcrnrlon=LOWER_LEFT_X_LON, # X axis for map is longitude
            llcrnrlat=LOWER_LEFT_Y_LAT, # Y axis for map is latitude
            urcrnrlon=UPPER_LEFT_X_LON, 
            urcrnrlat=UPPER_LEFT_Y_LAT,
            resolution="i") # c-crude, l-low, i-intermediate, h-high, f-full

m.shadedrelief() # makes it look pretty :)
m.drawcountries(linewidth=1.2)
m.drawrivers(color="lightblue")

# Locate Chernobyl
m.scatter(chernobyl_coords[0], chernobyl_coords[1], latlon=True, s=100, c='red', marker='^', alpha=1) # puts a red triangle at the (lon,lat) coordinate

#Locate Each Sensor:
m.scatter(sensor_longitudes, sensor_latitudes, latlon=True, s=100, c='blue', marker='^', alpha=1) 



plt.show()